## Kubeflow Fairing Overview

Key Activities

- Create model class 
- Train locally
- Train on Kubernetes cluster
- Train on Google AI Platform

#### Requirements

In [ ]:
%%writefile requirements.txt

tensorflow==2.1.0
tensorflow-datasets==2.1.0
kubeflow-fairing==0.7.1
google-cloud-storage==1.26.0

In [ ]:
!pip install -r requirements.txt --user

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

#### Model Class

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
tfds.disable_progress_bar()
import logging
from datetime import datetime

import os
import glob
from google.cloud import storage
import re
import logging

_GCS_PREFIX = "gs://"


logger = tf.get_logger()
logging.basicConfig(
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt="%Y-%m-%dT%H:%M:%SZ",
    level=logging.INFO)
print('Tensorflow-version: {0}'.format(tf.__version__))





class Storage(object):
    @staticmethod
    def upload(uri: str, out_dir: str = None) -> str:
        logging.info("Copying contents from %s to %s", uri, out_dir)

        if out_dir.startswith(_GCS_PREFIX):
            Storage._upload_gcs(uri, out_dir)
        else:
            raise Exception("Cannot recognize storage type for " + uri +
                            "\n'%s' are the current available storage type." %
                            (_GCS_PREFIX))

        logging.info("Successfully copied %s to %s", uri, out_dir)
        return out_dir
    
    @staticmethod
    def _upload_gcs(uri, out_dir: str):
        try:
            storage_client = storage.Client()
        except exceptions.DefaultCredentialsError:
            storage_client = storage.Client.create_anonymous_client()
        
        bucket_args = out_dir.replace(_GCS_PREFIX, "", 1).split("/", 1)
        bucket_name = bucket_args[0]
        gcs_path = bucket_args[1] if len(bucket_args) > 1 else ""
        bucket = storage_client.bucket(bucket_name)
        Storage.upload_local_directory_to_gcs(uri,bucket, gcs_path)
    
    @staticmethod
    def upload_local_directory_to_gcs(local_path, bucket, gcs_path):
        assert os.path.isdir(local_path)
        for local_file in glob.glob(local_path + '/**'):
            if not os.path.isfile(local_file):
                Storage.upload_local_directory_to_gcs(local_file, bucket, gcs_path + "/" + os.path.basename(local_file))
            else:
                remote_path = os.path.join(gcs_path, local_file[1 + len(local_path):])
                blob = bucket.blob(remote_path)
                blob.upload_from_filename(local_file)


class TensorflowModel(object):
    
    def __init__(self, bucket=None,export=True):
        self.model = None
        self.export_dir = "{0}/export/001".format(bucket)
        self.export = export
        
    # prepare data
    def prepare_data(self, batch_size=64, shuffle_size=1000):

        def scale(image, label):
            image = tf.cast(image, tf.float32)
            image /= 255
            return image, label

        # Split the training set into 80% and 20% for training and validation
        train_validation_split = tfds.Split.TRAIN.subsplit([8, 2])
        ((train_data, validation_data), test_data),info = tfds.load(name="fashion_mnist:1.0.0", 
                                                             split=(train_validation_split, tfds.Split.TEST),
                                                             as_supervised=True, with_info=True)


        print("Training data count : ", int(info.splits['train'].num_examples * 0.8))
        print("Validation data count : ", int(info.splits['train'].num_examples * 0.2))
        print("Test data count : ", int(info.splits['test'].num_examples))


        # create dataset to be used for training process
        train_dataset = train_data.map(scale).shuffle(shuffle_size).batch(batch_size).repeat().prefetch(tf.data.experimental.AUTOTUNE)
        val_dataset = validation_data.map(scale).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
        test_dataset = test_data.map(scale).batch(batch_size)

        return train_dataset, val_dataset, test_dataset
    
    # build model
    def build_model(self, learning_rate=0.001):
        # define model architecture
        model = tf.keras.Sequential([
          tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(28, 28, 1), name='x'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dense(10, activation='softmax')
          ])
        # compile model with loss, optimizer and accuracy 
        model.compile(
            loss=tf.keras.losses.sparse_categorical_crossentropy,
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            metrics=['accuracy'])
        return model
    
    
    # callback
    def get_callbacks(self):
        # callbacks 
        # folder to store current training logs
        logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

        class customLog(tf.keras.callbacks.Callback):
            def on_epoch_end(self, epoch, logs={}):
                logging.info('epoch: {}'.format(epoch + 1))
                logging.info('loss={}'.format(logs['loss']))
                logging.info('accuracy={}'.format(logs['accuracy']))
                logging.info('val_accuracy={}'.format(logs['val_accuracy']))
        callbacks = [
            tf.keras.callbacks.TensorBoard(logdir),
            customLog()
        ]
        return callbacks
    
    # train model
    def train(self):
        # Data extraction and processing
        # set variables
        BUFFER_SIZE = 10000
        BATCH_SIZE = 64

        train_dataset, val_dataset, test_dataset = self.prepare_data(batch_size=BATCH_SIZE, shuffle_size=BUFFER_SIZE)

        # Build Model
        TF_LEARNING_RATE = 0.001
        model = self.build_model(learning_rate=TF_LEARNING_RATE)
        model.summary()

        # train model
        TF_EPOCHS=10
        TF_STEPS_PER_EPOCHS=3
        #TF_STEPS_PER_EPOCHS = int(np.ceil(48000 / float(BATCH_SIZE)))  # number of training samples / batch size

        model.fit(train_dataset, 
                  epochs=TF_EPOCHS,
                  steps_per_epoch=3,
                  validation_data=val_dataset,
                  callbacks=self.get_callbacks())
        
        # evaluate model
        result = model.evaluate(test_dataset, steps=1)
        loss = result[0]
        accuracy = result[1]
        print("loss : {0} accuracy : {1}".format(loss, accuracy))
        
        
        # save and export model
        tf.saved_model.save(model, "/tmp/export")
        if self.export:
            Storage.upload("/tmp/export", self.export_dir)
            
        
    def predict(self, X):
        # use this method to setup prediction
        pass 
        


### Train locally

In [ ]:
TensorflowModel(export=False).train()

### Training on Kubernetes Cluster

In [ ]:
from kubeflow import fairing
import importlib
from kubeflow.fairing import TrainJob

In [ ]:
GCP_PROJECT = fairing.cloud.gcp.guess_project_name()
GCS_BUCKET_ID = "{}-fashion-mnist-fairing-k8s-backend".format(GCP_PROJECT)
GCS_BUCKET = "gs://{}".format(GCS_BUCKET_ID)
!gsutil rm -r {GCS_BUCKET}
!gsutil mb {GCS_BUCKET}

In [ ]:
GCP_PROJECT = fairing.cloud.gcp.guess_project_name()
DOCKER_REGISTRY = 'gcr.io/{}/fairing-job-mnist'.format(GCP_PROJECT)

Fairing supports multiple backend such as 

- KubeflowGKEBackend (Google: GKE Cluster)
- KubeflowAzureBackend (Azure)
- KubeflowAWSBackend (AWS)
- GKEManagedBackend (Google AI Platform)

In [ ]:
#set fairing backend
FAIRING_BACKEND = 'KubeflowGKEBackend'

BuildContext = None
BackendClass = getattr(importlib.import_module('kubeflow.fairing.backends'), FAIRING_BACKEND)
train_job = TrainJob(lambda : TensorflowModel(bucket=GCS_BUCKET,export=True), input_files=["requirements.txt"],
                     docker_registry=DOCKER_REGISTRY,
                     backend=BackendClass(build_context_source=BuildContext))

train_job.submit()